# 🚀 VEC系统 - RSU计算资源对比实验

## 📋 实验配置
- **实验类型**: RSU计算资源敏感性分析
- **训练轮次**: 1500 episodes
- **随机种子**: 42
- **预计时长**: 2-3小时（T4 GPU）

## ⚙️ 使用前准备
1. 菜单栏: **代码执行程序** → **更改运行时类型** → 选择 **T4 GPU**
2. 依次运行下面的单元格

In [ ]:
# 检查GPU
import torch
print(f'PyTorch: {torch.__version__}')
print(f'CUDA: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
else:
    print('⚠️ 请在菜单栏选择GPU运行时！')

# 挂载Google Drive
from google.colab import drive
import os
drive.mount('/content/drive')
save_dir = '/content/drive/MyDrive/VEC_results'
os.makedirs(save_dir, exist_ok=True)
print(f'✅ 结果保存目录: {save_dir}')

In [ ]:
# 方法1：从GitHub克隆项目（推荐）
import os

# 📌 修改这里：填入你的Git仓库地址
GIT_REPO_URL = 'https://github.com/YOUR_USERNAME/VEC_mig_caching.git'  # ← 修改为你的仓库地址

print(f'📦 正在克隆仓库: {GIT_REPO_URL}')
!git clone {GIT_REPO_URL} /content/VEC_mig_caching

# 进入项目目录
os.chdir('/content/VEC_mig_caching')
print(f'✅ 项目目录: {os.getcwd()}')

# 查看目录结构
!ls -la | head -15

In [ ]:
# 方法2：上传ZIP文件（如果没有Git仓库，使用此方法）
# 取消下面的注释来使用

# from google.colab import files
# import zipfile
# import shutil
# 
# print('📤 请选择VEC_mig_caching.zip文件上传...')
# uploaded = files.upload()
# 
# zip_name = list(uploaded.keys())[0]
# with zipfile.ZipFile(zip_name, 'r') as z:
#     z.extractall('/content')
# 
# project_dir = '/content/VEC_mig_caching'
# if not os.path.exists(project_dir):
#     for item in os.listdir('/content'):
#         if 'VEC' in item and os.path.isdir(f'/content/{item}'):
#             shutil.move(f'/content/{item}', project_dir)
#             break
# 
# os.chdir(project_dir)
# print(f'✅ 项目目录: {os.getcwd()}')
# !ls -la | head -15

In [ ]:
# 安装依赖
!pip install flask-socketio pyyaml -q

# 创建目录
!mkdir -p results/td3_strategy_suite academic_figures logs

# 验证文件
print('✅ 依赖安装完成\n')
print('📂 关键文件检查:')
files_check = [
    'experiments/td3_strategy_suite/run_bandwidth_cost_comparison.py',
    'requirements.txt',
    'config/system_config.py'
]
for f in files_check:
    print(f"{'✅' if os.path.exists(f) else '❌'} {f}")

In [ ]:
# 🚀 运行RSU计算资源对比实验（1500轮）
!python experiments/td3_strategy_suite/run_bandwidth_cost_comparison.py \
    --experiment-types rsu_compute \
    --rsu-compute-levels default \
    --episodes 1500 \
    --seed 42

In [ ]:
# 💡 快速验证模式（500轮，仅用于测试）
# !python experiments/td3_strategy_suite/run_bandwidth_cost_comparison.py \
#     --experiment-types rsu_compute \
#     --rsu-compute-levels default \
#     --episodes 500 \
#     --seed 42

In [ ]:
# 查看结果文件
print('📁 实验结果:')
!ls -lh results/td3_strategy_suite/ | grep rsu_compute

print('\n📊 生成图表:')
!ls -lh academic_figures/ | tail -10

# 显示图表
from IPython.display import Image, display
import glob

figures = sorted(glob.glob('academic_figures/*rsu_compute*.png'))
if figures:
    print(f'\n找到 {len(figures)} 张图表')
    for fig in figures[-3:]:
        print(f'\n📈 {os.path.basename(fig)}')
        display(Image(filename=fig))
else:
    print('⚠️ 未找到图表')

In [ ]:
# 显示关键指标
import json
import pandas as pd
import glob

summaries = sorted(glob.glob('results/td3_strategy_suite/*summary*.json'))
if summaries:
    with open(summaries[-1], 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    if 'rsu_compute' in data:
        print('🎯 RSU计算资源对比结果\n' + '='*80)
        for level, strategies in data['rsu_compute'].items():
            print(f'\n📌 {level}\n' + '-'*80)
            rows = []
            for strat, metrics in strategies.items():
                rows.append({
                    '策略': strat,
                    '时延(s)': f"{metrics.get('avg_delay', 0):.4f}",
                    '能耗(J)': f"{metrics.get('avg_energy_consumption', 0):.2f}",
                    '吞吐量': f"{metrics.get('avg_throughput_mbps', 0):.2f}",
                    'RSU利用率': f"{metrics.get('avg_rsu_utilization', 0):.2%}"
                })
            print(pd.DataFrame(rows).to_string(index=False))
    else:
        print('⚠️ 未找到RSU实验数据')
else:
    print('⚠️ 未找到摘要文件')

In [ ]:
# 打包并保存结果
from datetime import datetime
from google.colab import files

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
result_zip = f'rsu_results_{timestamp}'
figure_zip = f'rsu_figures_{timestamp}'

!zip -r {result_zip}.zip results/td3_strategy_suite/
!zip -r {figure_zip}.zip academic_figures/

print(f'✅ 已打包:')
print(f'   {result_zip}.zip')
print(f'   {figure_zip}.zip')

# 保存到Drive
if os.path.exists('/content/drive/MyDrive/VEC_results'):
    !cp {result_zip}.zip /content/drive/MyDrive/VEC_results/
    !cp {figure_zip}.zip /content/drive/MyDrive/VEC_results/
    print('\n📤 已保存到Google Drive')

# 下载到本地
print('\n📥 开始下载到本地...')
files.download(f'{result_zip}.zip')
files.download(f'{figure_zip}.zip')